# Division reduced dataset to 30 seconds mp3 files

## Import

In [5]:
import os
import librosa
import soundfile as sf
import pandas as pd
from tqdm import tqdm

## Creating directories

In [2]:
# Ścieżki
metadata_path = "../../datasets/jamendo/metadata/reduced_audio_dataset_14_genres.csv"
audio_folder = "../../datasets/jamendo/reduced_audio_500/"
output_folder = "../../datasets/jamendo/processed_audio/"
output_metadata_path = "../../datasets/jamendo/metadata/processed_audio_metadata.csv"

# Tworzenie folderu wyjściowego, jeśli nie istnieje
os.makedirs(output_folder, exist_ok=True)

# Wczytanie danych z metadanych
metadata = pd.read_csv(metadata_path)

# Lista do przechowywania nowych metadanych
new_metadata = []

## Processing audio files into 30-second files, removing silence, creating metadata

In [ ]:
# Funkcja do przetwarzania plików
for idx, row in tqdm(metadata.iterrows(), total=len(metadata), desc="Processing files"):
    audio_path = os.path.join(audio_folder, row["PATH"])
    try:
        # Wczytanie pliku audio
        audio, sr = librosa.load(audio_path, sr=None)
        
        # Usuwanie ciszy z początku i końca
        audio_trimmed, _ = librosa.effects.trim(audio, top_db=40)
        
        # Dzielenie na 30-sekundowe kawałki
        chunk_length = 30 * sr  # 30 sekund w próbkach
        for i, start_idx in enumerate(range(0, len(audio_trimmed), chunk_length)):
            chunk = audio_trimmed[start_idx:start_idx + chunk_length]
            if len(chunk) < chunk_length:
                break  # Pomijanie ostatniego zbyt krótkiego kawałka
            
            # Generowanie unikalnej nazwy pliku
            new_id = f"{row['ID']}_chunk_{i}"
            output_path = os.path.join(output_folder, f"{new_id}.wav")
            
            # Zapis kawałka
            sf.write(output_path, chunk, sr)
            
            # Dodanie do nowych metadanych
            new_metadata.append({
                "id": new_id,
                "original_id": row["ID"],
                "track_id": row["TRACK_ID"],
                "artist_id": row["ARTIST_ID"],
                "album_id": row["ALBUM_ID"],
                "path": output_path,
                "duration": len(chunk) / sr,  # w sekundach
                "tags": row["TAGS"],
                "genre_list": row["genre_list"],
                "genre_count": row["genre_count"]
            })
    except Exception as e:
        print(f"Nie udało się przetworzyć pliku {audio_path}: {e}")


Exception ignored in: <function tqdm.__del__ at 0x000001E6786A2EE0>
Traceback (most recent call last):
  File "c:\Users\szyme\Documents\Praca_Magisterska\Musical-Genres-Classification\.venv\lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "c:\Users\szyme\Documents\Praca_Magisterska\Musical-Genres-Classification\.venv\lib\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'



Nie udało się przetworzyć pliku ../../datasets/jamendo/reduced_audio_500/15/215.mp3: 'id'


Nie udało się przetworzyć pliku ../../datasets/jamendo/reduced_audio_500/00/1197000.mp3: 'id'


Nie udało się przetworzyć pliku ../../datasets/jamendo/reduced_audio_500/03/1197003.mp3: 'id'


Nie udało się przetworzyć pliku ../../datasets/jamendo/reduced_audio_500/19/219.mp3: 'id'


Nie udało się przetworzyć pliku ../../datasets/jamendo/reduced_audio_500/28/228.mp3: 'id'


Nie udało się przetworzyć pliku ../../datasets/jamendo/reduced_audio_500/31/231.mp3: 'id'


Processing files:   0%|          | 6/6750 [00:08<2:43:48,  1.46s/it]


KeyboardInterrupt: 

## Zapis metadanych

In [ ]:
# Tworzenie nowego pliku CSV z metadanymi
new_metadata_df = pd.DataFrame(new_metadata)
new_metadata_df.to_csv(output_metadata_path, index=False)
print(f"Nowy plik CSV z metadanymi został zapisany w: {output_metadata_path}")